In [1]:
# import os + os.chir is used to get to the llms directory to use the usermodules 
import os
os.chdir('C:/Users/AViey/OneDrive/Documents/College/UCI/Stats/stats170a/copilot/copilot/llms/user_modules') 

from  table_instances import Summaries, Sentiments, Base
from  db import Session, get_training_data, add_summaries_data
from helper_functions import compute_summary_scores
import pandas as pd

session = Session()

c:\Users\AViey\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
training = get_training_data()
training.head()

,dialogue_id,dataset,dialogue_text,actual_summary,actual_sentiment
0,3816,training,"#Person1#: Good morning, sir. Is there anythin...",#Person2# wants a suit made in 100 % black Bar...,neutral
1,3,training,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,positive
2,4,training,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,negative
3,5,training,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,positive
4,6,training,"#Person1#: Happy birthday, Aims!\n#Person2#: T...",Lisa gives Aims a birthday present and Aims lo...,positive


In [21]:
gemma = pd.read_csv('C:/Users/AViey/OneDrive/Documents/College/UCI/Stats/stats170a/copilot/copilot/Datasets/gemma_sample_scores.csv')
gemma.head()

,dialogue_id,actual_summary,generated_summary,rouge_1,rouge_2,rouge_l,bert_precision,bert_recall,bert_f1,meteor
0,3816,#Person2# wants a suit made in 100 % black Bar...,The dialogue is about a person ordering a suit...,0.468085,0.259259,0.468085,0.893794,0.931543,0.912278,0.669124
1,3,#Person1#'s looking for a set of keys and asks...,Two people are talking and searching for missi...,0.269231,0.053333,0.269231,0.867384,0.865039,0.866210,0.270274
2,4,#Person1#'s angry because #Person2# didn't tel...,"The dialogue is between two people, #Person1# ...",0.250000,0.043011,0.222222,0.862367,0.899115,0.880357,0.455599
3,5,Malik invites Nikki to dance. Nikki agrees if ...,"The dialogue is between two people, Malik and ...",0.156863,0.027778,0.156863,0.856515,0.872255,0.864313,0.231092
4,6,Lisa gives Aims a birthday present and Aims lo...,"The dialogue is between two people, Lisa and A...",0.341463,0.000000,0.243902,0.873825,0.910648,0.891857,0.306122


In [22]:
usage = pd.read_csv('C:/Users/AViey/OneDrive/Documents/College/UCI/Stats/stats170a/copilot/copilot/Datasets/gemma_summary_results.csv')
usage.head()

,dialogue_id,Time Taken (s),GPU Usage (MB),Summary
0,3816,0.633181,750.495742,The dialogue is about a person ordering a suit...
1,3,0.633181,750.495742,Two people are talking and searching for missi...
2,4,0.633181,750.495742,"The dialogue is between two people, #Person1# ..."
3,5,0.633181,750.495742,"The dialogue is between two people, Malik and ..."
4,6,0.633181,750.495742,"The dialogue is between two people, Lisa and A..."


In [23]:
usage = usage.rename(columns={'Summary': 'generated_summary', 'Time Taken (s)':'time_summary_taken', 'GPU Usage (MB)':'memory_summary_usage'})
usage.head()

,dialogue_id,time_summary_taken,memory_summary_usage,generated_summary
0,3816,0.633181,750.495742,The dialogue is about a person ordering a suit...
1,3,0.633181,750.495742,Two people are talking and searching for missi...
2,4,0.633181,750.495742,"The dialogue is between two people, #Person1# ..."
3,5,0.633181,750.495742,"The dialogue is between two people, Malik and ..."
4,6,0.633181,750.495742,"The dialogue is between two people, Lisa and A..."


In [25]:
merged_df = pd.merge(gemma, usage[['dialogue_id', 'time_summary_taken', 'memory_summary_usage']], on='dialogue_id', how='inner')
merged_df.head()

,dialogue_id,actual_summary,generated_summary,rouge_1,rouge_2,rouge_l,bert_precision,bert_recall,bert_f1,meteor,time_summary_taken,memory_summary_usage
0,3816,#Person2# wants a suit made in 100 % black Bar...,The dialogue is about a person ordering a suit...,0.468085,0.259259,0.468085,0.893794,0.931543,0.912278,0.669124,0.633181,750.495742
1,3,#Person1#'s looking for a set of keys and asks...,Two people are talking and searching for missi...,0.269231,0.053333,0.269231,0.867384,0.865039,0.866210,0.270274,0.633181,750.495742
2,4,#Person1#'s angry because #Person2# didn't tel...,"The dialogue is between two people, #Person1# ...",0.250000,0.043011,0.222222,0.862367,0.899115,0.880357,0.455599,0.633181,750.495742
3,5,Malik invites Nikki to dance. Nikki agrees if ...,"The dialogue is between two people, Malik and ...",0.156863,0.027778,0.156863,0.856515,0.872255,0.864313,0.231092,0.633181,750.495742
4,6,Lisa gives Aims a birthday present and Aims lo...,"The dialogue is between two people, Lisa and A...",0.341463,0.000000,0.243902,0.873825,0.910648,0.891857,0.306122,0.633181,750.495742


In [27]:
merged_df["model_id"] = "gemma 7b"

In [28]:
merged_df = merged_df[['dialogue_id', 'model_id', 'generated_summary', 'rouge_1', 'rouge_2', 'rouge_l', 'bert_precision', 'bert_recall', 'bert_f1', 'meteor', 'memory_summary_usage', 'time_summary_taken']]
merged_df.head()

,dialogue_id,model_id,generated_summary,rouge_1,rouge_2,rouge_l,bert_precision,bert_recall,bert_f1,meteor,memory_summary_usage,time_summary_taken
0,3816,gemma 7b,The dialogue is about a person ordering a suit...,0.468085,0.259259,0.468085,0.893794,0.931543,0.912278,0.669124,750.495742,0.633181
1,3,gemma 7b,Two people are talking and searching for missi...,0.269231,0.053333,0.269231,0.867384,0.865039,0.866210,0.270274,750.495742,0.633181
2,4,gemma 7b,"The dialogue is between two people, #Person1# ...",0.250000,0.043011,0.222222,0.862367,0.899115,0.880357,0.455599,750.495742,0.633181
3,5,gemma 7b,"The dialogue is between two people, Malik and ...",0.156863,0.027778,0.156863,0.856515,0.872255,0.864313,0.231092,750.495742,0.633181
4,6,gemma 7b,"The dialogue is between two people, Lisa and A...",0.341463,0.000000,0.243902,0.873825,0.910648,0.891857,0.306122,750.495742,0.633181


In [29]:
add_summaries_data(merged_df)